In [ ]:
import findspark
findspark.init('/home/anqi/spark-3.0.0-preview2-bin-hadoop2.7')

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext

In [ ]:
# create a class via namedtuple
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [ ]:
sc = SparkContext('local[2]','NetworkWordCount')
sqlContext = SQLContext(sc)

In [ ]:
ssc = StreamingContext(sc,10)

In [ ]:
lines = ssc.socketTextStream('localhost',9989)

In [ ]:
words = lines.flatMap(lambda line: line.split(' '))

In [ ]:
hashword = words.filter( lambda word: word.lower().startswith("#") )

In [ ]:
pairs = hashword.map(lambda hashwordd:(hashwordd.lower(),1))

In [ ]:
word_counts = pairs.reduceByKey(lambda num1,num2:num1+num2)

In [ ]:
twitterobj = word_counts.map( lambda rec: Tweet( rec[0], rec[1] ) )

In [ ]:
totable = twitterobj.foreachRDD(lambda rdd: rdd.toDF().sort(desc("count")).limit(2).registerTempTable("tweets"))
#totable = twitterobj.foreachRDD(lambda rdd: rdd.toDF())

In [ ]:
word_counts.pprint()

In [ ]:
twitterobj.pprint()

In [ ]:
ssc.start()

In [ ]:
top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )

In [ ]:
ssc.stop()

In [ ]:
twitterobj

In [ ]:
word_counts